In [57]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [59]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [70]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (307.00 seconds): Collecting URLs... theverge            

n_total     25626
n_unique    25565
dtype: int64

,pubs,actual_pub
https://businessinsider.co.za/boyfriend-stokvel-whatsapp-group-allows-you-to-advertise-your-ex-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/firing-of-chinese-facebook-engineer-prompts-anger-in-china-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/trump-fending-off-impeachment-with-greatest-economy-claims-economists-dispel-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/trump-ratchets-up-the-pressure-on-fox-news-ripping-it-apart-2019-10,"businessinsider, news24",businessinsider
https://cnbc.com/2019/10/11/trump-gives-his-administration-broad-powers-to-put-sanctions-on-turkey.html,"cnbc, realclearpolitics",cnbc
https://cnn.com/2019/10/11/media/fox-news-shepard-smith/index.html,"cnn, axios",cnn
https://foxnews.com/opinion/trump-hispanic-voters-economy-democrats-mercedes-schlapp,"fox, realclearpolitics",fox
https://hotair.com/archives/jazz-shaw/2019/10/12/cuomo-orders-utility-pump-imaginary-natural-gas,"townhall, hotair",hotair
https://msnbc.com/hallie-jackson/watch/appeals-court-upholds-democrats-subpoena-for-trump-financial-records-71067717523,"msnbc, nbcnews",msnbc
https://msnbc.com/the-beat-with-ari/watch/swizz-beatz-on-working-with-beyonce-jay-z-drake-and-upending-the-art-world-70266437589,"msnbc, nbcnews",msnbc


huffpost                  79
cnn                      844
investing.com            935
politico                 210
time                      21
cnbc                     163
fox                      337
bbc                      266
businessinsider          564
morningstar              102
wsj                      107
nytimes                  108
guardian                 158
reuters                 1158
washingtontimes          471
washingtonpost           129
cbs                      406
marketwatch              177
atlantic                 104
vice                     101
npr                      886
newrepublic               20
yahoo                    332
independent              711
heritage                 280
zdnet                    395
townhall                 615
abcnews                  103
hotair                    84
cbc                      291
                        ... 
metro                    251
msnbc                    105
nationalreview           741
news24        

In [71]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [72]:
select_results = crawler.select(); display(*select_results)

100.00% (48.19 seconds): Selecting URLs... cryptonews          

n_total     472
n_unique    472
dtype: int64

,pubs,actual_pub


huffpost                 0
cnn                      5
investing.com           15
politico                 1
time                     0
cnbc                     2
fox                     14
bbc                      2
businessinsider         15
morningstar              0
wsj                      1
nytimes                  3
guardian                10
reuters                 41
washingtontimes         11
washingtonpost           7
cbs                      3
marketwatch              3
atlantic                 0
vice                     0
npr                      0
newrepublic              0
yahoo                   51
independent              4
heritage                 0
zdnet                    0
townhall                 2
abcnews                  2
hotair                   3
cbc                      5
                        ..
metro                    9
msnbc                    5
nationalreview           0
news24                  10
techcrunch               0
arstechnica              0
e

In [73]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [74]:
download_results = crawler.download(); download_results

100.00% (548.29 seconds): Downloading... euronews            

,downloaded,trashed
abcnews,2,0
afr,0,0
aljazeera,2,15
americanconservative,0,0
arirang,0,0
arstechnica,0,0
arynews,2,1
atlantic,0,0
axios,6,3
bbc,2,0


In [14]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

[]